In [260]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql import functions as sqlf 
#col, lit, udf,sum,avg,max,min,mean,count, udf 
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, DoubleType, LongType

In [261]:
spark = SparkSession.builder.appName('Stage - Ingest').getOrCreate()
conf = SparkConf().setAppName('Stage - Ingest')
sc = SparkContext.getOrCreate(conf=conf)

In [262]:
import json
import logging
from pandas import DataFrame
import os
import pendulum
from datetime import datetime

def get_subdirectories(path: str) -> list:
    """
    Function that searchs for sub-directories inside a directory 
    then returns a list of all the directories names.
    """
    if path.split('/')[7] in ['customer', 'transaction']:
        subdirectories = [f.path for f in os.scandir(path) if f.is_dir()]
        subdirectories = [x.split('/')[8] for x in subdirectories]
        csv_directories = [directory.split('_')[1] for directory in subdirectories]
        return csv_directories
    else:
        subdirectories = [f.path for f in os.scandir(path) if f.is_dir()]
        subdirectories = [x.split('/')[7] for x in subdirectories]
        return [datetime.strptime(directory, '%Y-%m-%d').date() for directory in subdirectories] 
        

def get_latest_folder(folders: list) -> str:
    """
    Returns the latest date from the list.
    """
    try:
        return str(max(folders))
    except:
        return logging.critical('Couldn\'t find any sub-directory.')


def loadJsonData(json_path: str) -> DataFrame:
    """
    Function that returns a dataframe from a valid directory that contains jsonlines files.
    """
    latest = get_latest_folder(get_subdirectories(json_path))
    if os.path.exists(os.path.dirname(os.path.join(json_path, latest))):
        df = spark.read.json(os.path.join(json_path, latest))
        return df
    else:
        return logging.critical('Path to jsonl files doesn\'t exist')


def loadParquetData(parquet_path: str) -> DataFrame:
    """
    Function that returns a dataframe from a valid directory that contains parquet files.
    """
    latest = get_latest_folder(get_subdirectories(parquet_path))
    if os.path.exists(os.path.dirname(parquet_path)):
        df = spark.read.option('recursiveFileLookup', 'true').option('header', 'true').parquet(parquet_path)
        return df
    else:
        return logging.critical('Path to parquet file deosn\'t exist')


def loadCSVData(csv_path: str) -> DataFrame:
    """
    Function that returns a dataframe from a valid directory that contains csv files.
    """
    latest = get_latest_folder(get_subdirectories(csv_path))
    if os.path.exists(os.path.dirname(csv_path)):
        df = spark.read.option('recursiveFileLookup', 'true').option('header', 'true').csv(csv_path)
        return df
    else:
        return logging.critical('Path to csv file doesn\'t exist')

In [314]:
# Ingesting jsonlines data
df = loadJsonData(json_path='/Users/gonzo/Desktop/capstone_project/data_storage/json_storage/')
df = df.select('id', 'ts', 'customer_first_name', 'customer_last_name', 'amount', 'type')
# df.select(['id']).distinct().count()
df = df.repartition(2)
df.rdd.getNumPartitions()
# df.show()

2

In [315]:
# Ingesting parquet data
df2 = loadParquetData(parquet_path='/Users/gonzo/Desktop/capstone_project/data_storage/parquet_storage/')
df2 = df2.repartition(2)
df2.rdd.getNumPartitions()

2

In [316]:
# Ingesting RDBMS (PostgreSQL) data
df3 = loadCSVData(csv_path='/Users/gonzo/Desktop/capstone_project/data_storage/pgdata/customer/')
df3 = df3.repartition(2)
df3.rdd.getNumPartitions()

2

In [317]:
# Ingesting RDBMS (PostgreSQL) data
df4 = loadCSVData(csv_path='/Users/gonzo/Desktop/capstone_project/data_storage/pgdata/transaction/')
df4 = df4.repartition(2)
df4.rdd.getNumPartitions()

2

In [300]:
df.printSchema()

root
 |-- id: long (nullable = true)
 |-- ts: string (nullable = true)
 |-- customer_first_name: string (nullable = true)
 |-- customer_last_name: string (nullable = true)
 |-- amount: string (nullable = true)
 |-- type: long (nullable = true)



In [301]:
df2.printSchema()

root
 |-- Id: long (nullable = true)
 |-- First_name: string (nullable = true)
 |-- Last_name: string (nullable = true)
 |-- Amount: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- Store_id: long (nullable = true)



In [302]:
df3.printSchema()

root
 |-- id: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- phone_number: string (nullable = true)
 |-- address: string (nullable = true)



In [303]:
df4.printSchema()

root
 |-- id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- transaction_ts: string (nullable = true)
 |-- amount: string (nullable = true)



In [304]:
df2 = df2.withColumnRenamed('First_name', 'customer_first_name')
df2 = df2.withColumnRenamed('Last_name', 'customer_last_name')
df2 = df2.withColumnRenamed('Amount', 'amount')
df = df.withColumnRenamed('ts', 'timestamp')
df = df.withColumnRenamed('Store_id', 'store_id')
df2.show()

+---+-------------------+------------------+----------+-------------------+--------+
| Id|customer_first_name|customer_last_name|    amount|          timestamp|Store_id|
+---+-------------------+------------------+----------+-------------------+--------+
|256|              Blake|             Scott|     $3.48|2022-10-03T09:58:52|       4|
|283|             Andrew|           Russell|     $7.59|2022-10-02T07:21:14|      11|
|231|             Andrew|            Cooper|$11,380.13|2022-10-02T23:09:56|      10|
|799|           Kimberly|              Lara| $7,642.31|2022-10-03T07:26:19|      20|
|694|              Shane|             Allen|     $5.53|2022-10-04T07:26:22|       2|
|107|            Michael|          Thompson|    $11.94|2022-10-02T23:37:12|      17|
| 25|              Erika|             Green| $4,462.03|2022-10-03T15:37:49|       8|
|374|            Michael|             Clark|    $98.63|2022-10-02T12:34:23|      14|
|268|             Pamela|          Copeland|   $549.70|2022-10-02

In [305]:
# Parquet & Json
json_parquetDF = df.join(df2, ['id', 'customer_first_name', 'customer_last_name', 'timestamp', 'amount'], "fullouter")
json_parquetDF = json_parquetDF.select('id', 'type', 'store_id', 'amount', 'customer_first_name', 'customer_last_name', 'timestamp')
json_parquetDF.show()

+---+----+--------+----------+-------------------+------------------+-------------------+
| id|type|store_id|    amount|customer_first_name|customer_last_name|          timestamp|
+---+----+--------+----------+-------------------+------------------+-------------------+
|  2|   1|    null|     $4.18|             Brenda|           Sanchez|2022-10-02T19:27:13|
|  4|   0|    null|$23,199.16|               Dawn|              Hill|2022-10-03T02:55:52|
|  6|   0|    null|$84,143.57|            Jeffrey|          Gonzales|2022-10-04T01:20:01|
|  7|null|      17|   $711.35|             Thomas|            Oliver|2022-10-05T10:52:27|
| 10|null|      20|    $67.40|            William|             Reyes|2022-10-04T10:15:45|
| 11|null|       2| $4,516.14|             Jeremy|             Moody|2022-10-03T13:18:38|
| 19|null|      11|     $2.12|             Dustin|           Hancock|2022-10-03T16:55:46|
| 19|   0|    null|     $7.75|              Sarah|             Craig|2022-10-04T19:55:02|
| 23|null|

In [306]:
# Both dataframes from postgresql
postgresqlDF = df3.join(df4, ['id'], "fullouter")
postgresqlDF = postgresqlDF.select('id', 'customer_id', 'amount', 'first_name', 'last_name', 'phone_number', 'address', 'transaction_ts')
postgresqlDF.show(100)

+---+-----------+----------+-----------+----------+------------+-----------------+-------------------+
| id|customer_id|    amount| first_name| last_name|phone_number|          address|     transaction_ts|
+---+-----------+----------+-----------+----------+------------+-----------------+-------------------+
| 12|        705|    $37.96|   Patricia|  Williams|  1624390971|Port Benjaminfurt|2022-10-03 02:41:43|
|129|        837|    $27.51|    Tiffany|   Farrell|  0174130827|New Marthaborough|2022-10-03 10:40:03|
| 17|        530|$65,981.69|  Stephanie|    Chavez|  9952997214|    Michelleburgh|2022-10-02 15:00:14|
|170|        689| $4,249.06|     Arthur|    Hester|  9696686678|     West Nichole|2022-10-03 08:34:01|
| 18|        363|   $654.36|     Denise|  Anderson|  5718166161|      Melissafurt|2022-10-03 12:56:18|
|199|        666|$12,611.06|       Lynn|   Swanson|  1308999765|        Erikville|2022-10-03 20:20:26|
|218|        329|   $381.18|   Kimberly|     Baker|  5596293609|         

In [307]:
json_parquetDF.printSchema()

root
 |-- id: long (nullable = true)
 |-- type: long (nullable = true)
 |-- store_id: long (nullable = true)
 |-- amount: string (nullable = true)
 |-- customer_first_name: string (nullable = true)
 |-- customer_last_name: string (nullable = true)
 |-- timestamp: string (nullable = true)



In [308]:
postgresqlDF = postgresqlDF.withColumnRenamed('first_name', 'customer_first_name')
postgresqlDF = postgresqlDF.withColumnRenamed('last_name', 'customer_last_name')
postgresqlDF = postgresqlDF.withColumnRenamed('transaction_ts', 'timestamp')
"""
Change id str -> long
customer_id str -> long
"""
postgresqlDF = postgresqlDF.withColumn('id', postgresqlDF['id'].cast(LongType()))
postgresqlDF = postgresqlDF.withColumn('customer_id', postgresqlDF['id'].cast(LongType()))
postgresqlDF.printSchema()

root
 |-- id: long (nullable = true)
 |-- customer_id: long (nullable = true)
 |-- amount: string (nullable = true)
 |-- customer_first_name: string (nullable = true)
 |-- customer_last_name: string (nullable = true)
 |-- phone_number: string (nullable = true)
 |-- address: string (nullable = true)
 |-- timestamp: string (nullable = true)



In [561]:
unified_model = json_parquetDF.join(postgresqlDF, ['id', 'amount', 'customer_first_name', 'customer_last_name', 'timestamp'], 'fullouter')
unified_model = unified_model.select('id', 'customer_id', 'store_id', 'type', 'amount', 'customer_first_name', 'customer_last_name', 'phone_number', 'address', 'timestamp')
# unified_model.show(300)

In [319]:

# unified_model.coalesce(1)
unified_model = unified_model.repartition(2)
unified_model.write.csv('/Users/gonzo/Desktop/capstone_project/data_storage/storage/test', header=True)

In [353]:
unified_model.printSchema()

root
 |-- id: long (nullable = true)
 |-- customer_id: long (nullable = true)
 |-- store_id: long (nullable = true)
 |-- type: long (nullable = true)
 |-- amount: string (nullable = true)
 |-- customer_first_name: string (nullable = true)
 |-- customer_last_name: string (nullable = true)
 |-- phone_number: string (nullable = true)
 |-- address: string (nullable = true)
 |-- timestamp: string (nullable = true)



In [562]:
unified_model.repartition(2).rdd.getNumPartitions()

2

In [563]:
"""
Transform amount str -> float
"""
def transform_amount(x) -> list:
    return float(x[1:].replace(',', ''))

transformUDF = sqlf.udf(lambda x : transform_amount(x), FloatType())
test_df = unified_model.withColumn('float_amount', transformUDF(unified_model.amount))
unified_model = test_df.select('id', 'customer_id', 'store_id', 'type', 'float_amount', 'customer_first_name', 'customer_last_name', 'phone_number', 'address', 'timestamp')
unified_model.show()

+---+-----------+--------+----+------------+-------------------+------------------+------------+-----------------+-------------------+
| id|customer_id|store_id|type|float_amount|customer_first_name|customer_last_name|phone_number|          address|          timestamp|
+---+-----------+--------+----+------------+-------------------+------------------+------------+-----------------+-------------------+
|  2|       null|    null|   1|        4.18|             Brenda|           Sanchez|        null|             null|2022-10-02T19:27:13|
|  4|       null|    null|   0|    23199.16|               Dawn|              Hill|        null|             null|2022-10-03T02:55:52|
|  6|       null|    null|   0|    84143.57|            Jeffrey|          Gonzales|        null|             null|2022-10-04T01:20:01|
|  7|       null|      17|null|      711.35|             Thomas|            Oliver|        null|             null|2022-10-05T10:52:27|
| 10|       null|      20|null|        67.4|           

In [564]:
unified_model.count()

400

# Count and sum amount transactions for each type for day - online

In [391]:
online = unified_model.filter(unified_model.type == 1).groupBy(unified_model.type).agg(sqlf.sum('float_amount').alias('Total'), sqlf.count('*').alias('Total_transactions'))
online.show()

+----+-----------------+------------------+
|type|            Total|Total_transactions|
+----+-----------------+------------------+
|   1|507014.9718411267|                46|
+----+-----------------+------------------+



# Count and sum amount transactions for each type for day - offline

In [392]:
offline = unified_model.filter(unified_model.type == 0).groupBy(unified_model.type).agg(sqlf.sum('float_amount').alias('Total'), sqlf.count('*').alias('Total_transactions'))
offline.show()

+----+------------------+------------------+
|type|             Total|Total_transactions|
+----+------------------+------------------+
|   0|416384.93764860183|                54|
+----+------------------+------------------+



# Count and sum transaction for each store for day

In [393]:
stores_metrics = unified_model.groupBy(unified_model.store_id).agg(sqlf.count('*').alias('Number_Transactions'), sqlf.sum('float_amount').alias('Total_transactions'))
stores_metrics.show()

+--------+-------------------+------------------+
|store_id|Number_Transactions|Total_transactions|
+--------+-------------------+------------------+
|      19|                  7|159202.05113983154|
|       7|                  8| 17103.25972723961|
|    null|                200|2108326.1374286786|
|       6|                 13|178035.97977387905|
|       9|                 12|127308.06613391638|
|      17|                 11| 92913.98154425621|
|       5|                  8|  203422.657913208|
|       1|                 12| 274127.8973016739|
|      10|                 11|133775.20075827837|
|       3|                  7|155361.42032670975|
|      12|                 11|182003.42019462585|
|       8|                 14|324316.81068754196|
|      11|                 11|192654.62814463675|
|       2|                  9| 83981.44904321432|
|       4|                  8| 58352.48837661743|
|      13|                  5|10794.219762563705|
|      18|                 10| 9224.699713587761|


# Count and sum amount transactions for each city for day

In [397]:
cities_transactions = unified_model.groupBy('address').agg(sqlf.count('*').alias('Total'), sqlf.sum(unified_model.float_amount).alias('Transactions'))
cities_transactions.show()

+-----------------+-----+------------------+
|          address|Total|      Transactions|
+-----------------+-----+------------------+
|       Robertland|    4|  76071.6396484375|
|         Allenton|    3|17870.060585021973|
|             null|  300|3477939.7497224137|
|      Lake Karina|    7|159957.65662384033|
|     Edwardsburgh|    2| 622.5600242614746|
|         Lanefurt|    5|29834.430786132812|
|    Port Loriview|    3| 27612.91039276123|
|New Marthaborough|    3| 36311.05953025818|
|        Smithberg|    7|  66597.2815989852|
|      Veronicaton|    7| 46609.33814525604|
|    Michelleburgh|    4|119296.95797157288|
|        Hicksview|    3| 4204.259979248047|
|      Melissafurt|    6|  76940.2188911438|
|  Lake Jamesville|    2|3209.6200561523438|
|     Brownchester|    7| 50072.25079250336|
|Port Benjaminfurt|    9|  238838.906229496|
|        Erikville|    7| 24468.70987224579|
|     West Nichole|    6|118525.74819087982|
|       Averymouth|    6| 19814.55973815918|
|         

BACKFILLING ALL THE NULL VALUES FROM THE COLUMNS

In [558]:
from faker import Faker
import random

fake = Faker()

def backfillType(x) -> int:
    if x is None:
        return random.randint(0,1)
    else:
        return x    

def backfillStoreID(x) -> int:
    if x is None:
        return random.randint(1,20)
    else:
        return x

def backfillPhoneNumber(x) -> int:
    fake = Faker()
    if x is None:
        return fake.msisdn()[3:]
    else:
        return x

def backfillAddress(x) -> str:
    if x is None:
        return random.choice(['Allenton', 'Hicksview', 'Smithberg', 'Robertland', 'Veronicaton', 'Lake Jamesville', 'Port Benjaminfurt', 'Averymouth', 'Erikville', 'Port Loriview', 'Grahamstad', 'Edwardsburgh', 'New Marthaborough', 'Melissafurt', 'Lanefurt', 'Clayview', 'West Nichole', 'Brownchester', 'Lake Karina', 'Michelleburgh'])
    else:
        return x

def backfillCustomerID(x) -> int:
    if x is None:
        return random.randint(1, 900)
    else:
        return x

backfillType_udf = sqlf.udf(lambda x : backfillType(x), LongType())
backfillStoreId_udf = sqlf.udf(lambda x : backfillStoreID(x), IntegerType())
backfillPhoneNumber_udf = sqlf.udf(lambda x: backfillPhoneNumber(x), StringType())
backfillAddress_udf = sqlf.udf(lambda x: backfillAddress(x), StringType())
backfillCustomerID_udf = sqlf.udf(lambda x: backfillCustomerID(x), LongType())

backfilledDF = unified_model
backfilledDF.repartition(2)
backfilledDF = backfilledDF.withColumn('f_type', backfillType_udf(unified_model.type))
backfilledDF = backfilledDF.withColumn('f_store_id', backfillStoreId_udf(backfilledDF.store_id))
backfilledDF = backfilledDF.withColumn('f_phone_number', backfillPhoneNumber_udf(backfilledDF.phone_number))
backfilledDF = backfilledDF.withColumn('f_address', backfillAddress_udf(backfilledDF.address))
backfilledDF = backfilledDF.withColumn('f_customer_id', backfillCustomerID_udf(backfilledDF.customer_id))
backfilledDF = backfilledDF.select('id', 'f_customer_id', 'f_store_id', 'f_type', 'float_amount', 'customer_first_name', 'customer_last_name', 'f_phone_number', 'f_address', 'timestamp')
# backfilledDF.show()

+---+-------------+----------+------+------------+-------------------+------------------+--------------+-----------------+-------------------+
| id|f_customer_id|f_store_id|f_type|float_amount|customer_first_name|customer_last_name|f_phone_number|        f_address|          timestamp|
+---+-------------+----------+------+------------+-------------------+------------------+--------------+-----------------+-------------------+
|  2|          815|         6|     1|        4.18|             Brenda|           Sanchez|    7904112847|       Grahamstad|2022-10-02T19:27:13|
|  4|          791|        13|     0|    23199.16|               Dawn|              Hill|    8456415789|New Marthaborough|2022-10-03T02:55:52|
|  6|          147|        19|     0|    84143.57|            Jeffrey|          Gonzales|    5167666735|Port Benjaminfurt|2022-10-04T01:20:01|
|  7|          101|        17|     1|      711.35|             Thomas|            Oliver|    3303626529|Port Benjaminfurt|2022-10-05T10:52:27|